<h2> ta </h2>


In [ ]:
import pandas as pd

train_df = pd.read_csv("01_train.csv")

train_df.head()

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

# 당뇨병 데이터 분리
X = train_df.drop("Y", axis=1)
y = train_df["Y"]

# 훈련, 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])


# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=32)

# RMSE 확인
y_pred_val = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
print("Validation RMSE:", rmse)

# 모델 저장
model.save(".h5")

In [ ]:
test_df = pd.read_csv("01_test_x.csv")

test_df_scaled = scaler.transform(test_df)

test_predictions = model.predict(test_df_scaled)

test_df["Y"] = test_predictions

test_df.to_csv(".csv", index=False)